<a href="https://colab.research.google.com/github/VamsiMuraparthi/VamsiMuraparthi/blob/main/potato_downloaded_internship_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Give the input file name here
input_file="/content/211McCain.csv"
f_ind=input_file.index(".csv")
print(f_ind)
file_name = input_file[0:f_ind]
print(file_name)

18
/content/211McCain


In [5]:
import pandas as pd
output_file = file_name+"_output.csv"
df = pd.read_csv(input_file, skiprows=4, names=['Col1', 'Col2', 'x'])

In [6]:
df.dropna(how="all", inplace=True)
df=df.reset_index(drop=True)

In [7]:
data_index = df[df['Col1'] == 'SENSOR:'].index
data_index

Int64Index([0, 1692, 3384, 5076, 6768, 8460, 10150], dtype='int64')

In [8]:
for idx in data_index:
    df['Col2'][idx] = df['Col2'][idx] + ' ' + df['x'][idx]

df.drop(['x'], axis=1, inplace=True)
dfm = df

In [9]:
# Delete all rows where 'Col1' has value=VALUE 
rows_to_del = dfm[dfm['Col1'] == 'VALUE'].index
dfm.drop(rows_to_del, inplace=True)
dfm = dfm.reset_index(drop=True)

In [10]:
# __Finding indices of rows where we have sensor parameter name__
data_index = dfm[dfm['Col1'] == 'SENSOR:'].index
data_index

Int64Index([0, 1691, 3382, 5073, 6764, 8455, 10144], dtype='int64')

In [11]:
dfm.head(724)

,Col1,Col2
0,SENSOR:,Rain Fall mm
1,0,10-02-2022 22:40
2,0,10-02-2022 22:35
3,0,10-02-2022 22:33
4,0,10-02-2022 18:54
...,...,...
719,0,21-01-2022 21:37
720,0,21-01-2022 19:37
721,0,21-01-2022 17:36
722,0,21-01-2022 15:35


In [12]:
rows_to_del = [indx - 1 for indx in range(1, len(data_index)) if data_index[
    indx] == data_index[indx - 1] + 1]

dfm.drop(rows_to_del, inplace=True)
dfm = dfm.reset_index(drop=True)
data_index_new = dfm[dfm['Col1'] == 'SENSOR:'].index
data_index_new

Int64Index([0, 1691, 3382, 5073, 6764, 8455, 10144], dtype='int64')

In [13]:
column_names = dfm[dfm['Col1'] == 'SENSOR:'].Col2.tolist()

In [14]:
data_lst = []
size = len(data_index_new)

for i in range(size - 1):    
    data_lst.append(dfm[data_index_new[i] + 1:data_index_new[i + 1]][
                        'Col2'].reset_index(drop=True))
    data_lst.append(dfm[data_index_new[i] + 1:data_index_new[i + 1]][
                        'Col1'].reset_index(drop=True))

data_lst.append(
    dfm[data_index_new[size - 1] + 1:]['Col2'].reset_index(drop=True))
data_lst.append(
    dfm[data_index_new[size - 1] + 1:]['Col1'].reset_index(drop=True))

final_df = pd.DataFrame(list(zip(data_lst[0], data_lst[1])), columns=[
    'Timestamp', column_names[0]])

idx = 2

for i in range(1, size):
    new_df1 = pd.DataFrame(list(zip(data_lst[idx], data_lst[idx + 1])),
                           columns=['Timestamp', column_names[i]])
    final_df = final_df.join(new_df1.set_index('Timestamp'), on='Timestamp',
                             how='outer')
    idx = idx + 2
    
#Removing duplicate lines
final_df.drop_duplicates(subset=['Timestamp'], keep='first', inplace=True)

In [15]:
final_df.shape

(1679, 8)

In [16]:
#find index of battery column and then drop it
battery_col = -1
for i in range(len(column_names)):
    if 'battery' in column_names[i].lower():
        battery_col = i
        break
final_df.drop([column_names[battery_col]], axis=1, inplace=True)

In [17]:
column_names = final_df.columns
print(column_names)

final_df = final_df[~(final_df[column_names[1]].isna() & final_df[
    column_names[2]].isna() & final_df[column_names[3]].isna() & final_df[
    column_names[4]].isna())]

Index(['Timestamp', 'Rain Fall mm', 'Ambient relative humidity %RH',
       'Ambient temperature Deg Centigrade', 'Ultraviolet index unit',
       'Wind velocity km/h', 'Wind direction degree'],
      dtype='object')


In [18]:
final_df.to_csv(output_file, index=False)

In [19]:
final_df

,Timestamp,Rain Fall mm,Ambient relative humidity %RH,Ambient temperature Deg Centigrade,Ultraviolet index unit,Wind velocity km/h,Wind direction degree
0,10-02-2022 22:40,0,64.5,15,0,1.1,45
1,10-02-2022 22:35,0,64.5,15,0,1.1,45
2,10-02-2022 22:33,0,64.5,15,0,1.1,45
3,10-02-2022 18:54,0,59.8,18,0,0,270
4,10-02-2022 18:39,0,59.8,18,0,0,270
...,...,...,...,...,...,...,...
1685,29-10-2021 22:16,0,68.5,16,0,0,270
1686,29-10-2021 20:15,0,71.4,18,0,0.2,270
1687,29-10-2021 18:14,0,41,24,0,0.7,270
1688,29-10-2021 16:20,0,22.7,31,0,3.2,270


In [20]:
ts_col = final_df[column_names[0]]
ts_col

0       10-02-2022 22:40
1       10-02-2022 22:35
2       10-02-2022 22:33
3       10-02-2022 18:54
4       10-02-2022 18:39
              ...       
1685    29-10-2021 22:16
1686    29-10-2021 20:15
1687    29-10-2021 18:14
1688    29-10-2021 16:20
1689    29-10-2021 16:15
Name: Timestamp, Length: 1679, dtype: object

In [21]:
#time_ranges = ['00-02', '02-04', '04-06', '06-08', '08-10', '10-12', '12-14',
               #'14-16','16-18','18-20','20-22','22-24']
date_col = []
time_col = []
time_range = []

for ts in ts_col:
    tmp = ts.strip()
    tmp = tmp.split(' ')
    date_col.append(tmp[0])
    time_col.append(tmp[1])
    hour = tmp[1].split(':')[0]
    time_range.append(hour)

final_df.drop([column_names[0]], axis=1, inplace=True)
final_df.insert(0, "Date", date_col, True)
final_df.insert(1, "Time", time_col, True)
final_df.insert(2, "Time_Slot", time_range, True)

final_df

,Date,Time,Time_Slot,Rain Fall mm,Ambient relative humidity %RH,Ambient temperature Deg Centigrade,Ultraviolet index unit,Wind velocity km/h,Wind direction degree
0,10-02-2022,22:40,22,0,64.5,15,0,1.1,45
1,10-02-2022,22:35,22,0,64.5,15,0,1.1,45
2,10-02-2022,22:33,22,0,64.5,15,0,1.1,45
3,10-02-2022,18:54,18,0,59.8,18,0,0,270
4,10-02-2022,18:39,18,0,59.8,18,0,0,270
...,...,...,...,...,...,...,...,...,...
1685,29-10-2021,22:16,22,0,68.5,16,0,0,270
1686,29-10-2021,20:15,20,0,71.4,18,0,0.2,270
1687,29-10-2021,18:14,18,0,41,24,0,0.7,270
1688,29-10-2021,16:20,16,0,22.7,31,0,3.2,270


In [22]:
output_file_with_timeslots=file_name+"_output_with_timeslots.csv"
final_df.to_csv(output_file_with_timeslots, index=False)

In [23]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1679 entries, 0 to 1689
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Date                                1679 non-null   object
 1   Time                                1679 non-null   object
 2   Time_Slot                           1679 non-null   object
 3   Rain Fall mm                        1679 non-null   object
 4   Ambient relative humidity %RH       1679 non-null   object
 5   Ambient temperature Deg Centigrade  1679 non-null   object
 6   Ultraviolet index unit              1679 non-null   object
 7   Wind velocity km/h                  1677 non-null   object
 8   Wind direction degree               1678 non-null   object
dtypes: object(9)
memory usage: 131.2+ KB


In [24]:
final_df.isnull().sum()

Date                                  0
Time                                  0
Time_Slot                             0
Rain Fall mm                          0
Ambient relative humidity %RH         0
Ambient temperature Deg Centigrade    0
Ultraviolet index unit                0
Wind velocity km/h                    2
Wind direction degree                 1
dtype: int64

In [25]:
input_file=output_file_with_timeslots
df=pd.read_csv(input_file,header=0)
df.dropna(how="all", inplace=True)
df.dtypes

Date                                   object
Time                                   object
Time_Slot                               int64
Rain Fall mm                          float64
Ambient relative humidity %RH         float64
Ambient temperature Deg Centigrade      int64
Ultraviolet index unit                  int64
Wind velocity km/h                    float64
Wind direction degree                 float64
dtype: object

In [26]:
print(df.shape)
print(df.iloc[0])

(1679, 9)
Date                                  10-02-2022
Time                                       22:40
Time_Slot                                     22
Rain Fall mm                                 0.0
Ambient relative humidity %RH               64.5
Ambient temperature Deg Centigrade            15
Ultraviolet index unit                         0
Wind velocity km/h                           1.1
Wind direction degree                       45.0
Name: 0, dtype: object


In [27]:
columns_to_be_imputed=column_names[3:len(column_names)]
columns_to_be_imputed

Index(['Ambient temperature Deg Centigrade', 'Ultraviolet index unit',
       'Wind velocity km/h', 'Wind direction degree'],
      dtype='object')

In [28]:
from datetime import datetime
print(columns_to_be_imputed)
df["Date"] = pd.to_datetime(df["Date"],format='%d-%m-%Y') #'%d-%m-%Y') #'%Y-%m-%d')  #'%d-%m-%Y') #'%Y-%m-%d') #format='%Y-%m-%d')
df.dtypes

Index(['Ambient temperature Deg Centigrade', 'Ultraviolet index unit',
       'Wind velocity km/h', 'Wind direction degree'],
      dtype='object')


Date                                  datetime64[ns]
Time                                          object
Time_Slot                                      int64
Rain Fall mm                                 float64
Ambient relative humidity %RH                float64
Ambient temperature Deg Centigrade             int64
Ultraviolet index unit                         int64
Wind velocity km/h                           float64
Wind direction degree                        float64
dtype: object

In [29]:
from datetime import datetime, timedelta
for c in columns_to_be_imputed:
    l=df[df[c].isnull()].index.tolist()
    l.reverse()
    print(l)
    
    for i in l[0:]:
        
        if(i==df.shape[0]-1):
            
            u=df.loc[i-1,c]
            
            if(not(pd.isnull(u))):
                df.loc[i,c]=u
                print(df.loc[i,c])
            continue
        if(i!=0):
            u=df.loc[i-1,c]
        else:
            u=i
        l=df.loc[i+1,c]
        
        
        d=df.loc[i,"Date"]
        
        prev_date=(d-timedelta(days=1))
        #print("d=",d,"prev_date=",prev_date)
        prev_date=datetime(prev_date.year, prev_date.month,prev_date.day)
      
       
        time_slot1=df.loc[i,"Time_Slot"]
        
       
        
        
        prev_date_list=df.index[(df["Date"]==prev_date) & (df["Time_Slot"]==time_slot1)].tolist()
        k_flag=False
        
        
        if(len(prev_date_list)>=1):
            k=prev_date_list[0]
            
            k_flag=not(pd.isnull(df.loc[k,c]))
        
        
        if(k_flag):
            prev_date_val=df.loc[k,c]          
            l_flag=not(pd.isnull(l))
            r_flag=not(pd.isnull(u))
            
            print("l=",l,"u=",u,"k=",df.loc[k,c])
            if(l_flag and r_flag):
                df.loc[i,c]=(l+u+df.loc[k,c])/3.0;
                
            elif(l_flag and not(r_flag)):
                df.loc[i,c]=(l+df.loc[k,c])/2.0;
                
            elif(r_flag and not(l_flag)):
                df.loc[i,c]=(u+df.loc[k,c])/2;
                
        else:
            
            l_flag=not(pd.isnull(l))
            r_flag=not(pd.isnull(u))
            if(l_flag and r_flag):
                df.loc[i,c]=(l+u)/2.0;
                
            elif(l_flag and not(r_flag)):
                df.loc[i,c]=l;
                
            elif(r_flag and not(l_flag)):
                df.loc[i,c]=u;

[]
[]
[1463, 1408]
l= 3.9 u= 4.6 k= 3.0
[1463]


In [30]:
#f=file_path.index(".csv")
#print(f)
#output_file = file_path[0:f-1]+"_imputed.csv"
output_file=file_name+"_imputed.csv"
df.to_csv(output_file, index=False)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1679 entries, 0 to 1678
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Date                                1679 non-null   datetime64[ns]
 1   Time                                1679 non-null   object        
 2   Time_Slot                           1679 non-null   int64         
 3   Rain Fall mm                        1679 non-null   float64       
 4   Ambient relative humidity %RH       1679 non-null   float64       
 5   Ambient temperature Deg Centigrade  1679 non-null   int64         
 6   Ultraviolet index unit              1679 non-null   int64         
 7   Wind velocity km/h                  1679 non-null   float64       
 8   Wind direction degree               1679 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(1)
memory usage: 118.2+ KB


In [32]:
df=pd.read_csv(output_file,header=0)
df.info()
df.dropna(how="all", inplace=True)
df.info()
df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1679 entries, 0 to 1678
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Date                                1679 non-null   object 
 1   Time                                1679 non-null   object 
 2   Time_Slot                           1679 non-null   int64  
 3   Rain Fall mm                        1679 non-null   float64
 4   Ambient relative humidity %RH       1679 non-null   float64
 5   Ambient temperature Deg Centigrade  1679 non-null   int64  
 6   Ultraviolet index unit              1679 non-null   int64  
 7   Wind velocity km/h                  1679 non-null   float64
 8   Wind direction degree               1679 non-null   float64
dtypes: float64(4), int64(3), object(2)
memory usage: 118.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1679 entries, 0 to 1678
Data columns (total 9 columns):
 #   Colu

Date                                   object
Time                                   object
Time_Slot                               int64
Rain Fall mm                          float64
Ambient relative humidity %RH         float64
Ambient temperature Deg Centigrade      int64
Ultraviolet index unit                  int64
Wind velocity km/h                    float64
Wind direction degree                 float64
dtype: object

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1679 entries, 0 to 1678
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Date                                1679 non-null   object 
 1   Time                                1679 non-null   object 
 2   Time_Slot                           1679 non-null   int64  
 3   Rain Fall mm                        1679 non-null   float64
 4   Ambient relative humidity %RH       1679 non-null   float64
 5   Ambient temperature Deg Centigrade  1679 non-null   int64  
 6   Ultraviolet index unit              1679 non-null   int64  
 7   Wind velocity km/h                  1679 non-null   float64
 8   Wind direction degree               1679 non-null   float64
dtypes: float64(4), int64(3), object(2)
memory usage: 118.2+ KB


In [34]:
def Direction(x):
    if((x>=348.75 and x<=360) or (x>=0.0 and x<11.25)):
        return "N"
    elif(x>=11.25 and x<33.75):
        return "NNE"
    elif(x>=33.75 and x<56.25):
        return "NE"
    elif(x>=56.25 and x<=78.75):
        return "ENE"
    elif(x>78.75 and x<=101.25):
        return "E"
    elif(x>101.25 and x<=123.75):
        return "ESE"
    elif(x>123.75 and x<=146.25):
        return "SE"
    elif(x>146.25 and x<=168.75):
        return "SSE"
    elif(x>168.75 and x<=191.25):
        return "S"
    elif(x>191.25 and x<=213.75):
        return "SSW"
    elif(x>213.75 and x<=236.25):
        return "SW"
    elif(x>236.25 and x<=258.75):
        return "WSW"
    elif(x>258.75 and x<=281.25):
        return "W"
    elif(x>281.25 and x<=303.75):
        return "WNW"
    elif(x>303.75 and x<=326.25):
        return "NW"
    elif(x>326.25 and x<=348.75):
        return "NNW"
    else:
        print(x)
     
df['Wind Cardinal Direction'] = df['Wind direction degree'].apply(lambda x: Direction(x))
df.drop(columns="Wind direction degree",inplace=True)

In [35]:
print(df['Wind Cardinal Direction'])
df['Wind Cardinal Direction'].isnull().sum()

0       NE
1       NE
2       NE
3        W
4        W
        ..
1674     W
1675     W
1676     W
1677     W
1678     W
Name: Wind Cardinal Direction, Length: 1679, dtype: object


0

In [36]:
grouped_df=df.groupby(['Date','Time_Slot'])

In [37]:
print(grouped_df.head())
#print(grouped_df.agg(lambda x: pd.Series.mode(x,dropN)[0])

            Date   Time  Time_Slot  Rain Fall mm  \
0     2022-02-10  22:40         22           0.0   
1     2022-02-10  22:35         22           0.0   
2     2022-02-10  22:33         22           0.0   
3     2022-02-10  18:54         18           0.0   
4     2022-02-10  18:39         18           0.0   
...          ...    ...        ...           ...   
1674  2021-10-29  22:16         22           0.0   
1675  2021-10-29  20:15         20           0.0   
1676  2021-10-29  18:14         18           0.0   
1677  2021-10-29  16:20         16           0.0   
1678  2021-10-29  16:15         16           0.0   

      Ambient relative humidity %RH  Ambient temperature Deg Centigrade  \
0                              64.5                                  15   
1                              64.5                                  15   
2                              64.5                                  15   
3                              59.8                                  18   


In [38]:
df2=grouped_df.agg({df.columns.values[3]:['mean','min','max']})
df4=pd.DataFrame()
for col in df.columns.values[4:]:
    if(col!="Wind Cardinal Direction"):
        df3=grouped_df.agg({col:['mean','min','max']})
        df2=df2.join(df3)
    else:
        #df4["Wind Cardinal Direction","mode"]=grouped_df['Wind Cardinal Direction'].agg(lambda x:x.value_counts().index[0])
        df4["Wind Cardinal Direction","mode"]=grouped_df['Wind Cardinal Direction'].agg(lambda x: pd.Series.mode(x).iloc[0])
        print(df4.head())
        df2=df2.join(df4)
print(df2.head())


print("df2 size= ",df2.shape)

df2.to_csv("Others.csv")

df2.reset_index(inplace=True)
df2.info()

                     (Wind Cardinal Direction, mode)
Date       Time_Slot                                
2021-10-29 16                                      W
           18                                      W
           20                                      W
           22                                      W
2021-10-30 15                                      W
                     Rain Fall mm           Ambient relative humidity %RH  \
                             mean  min  max                          mean   
Date       Time_Slot                                                        
2021-10-29 16                 0.0  0.0  0.0                         23.95   
           18                 0.0  0.0  0.0                         41.00   
           20                 0.0  0.0  0.0                         71.40   
           22                 0.0  0.0  0.0                         68.50   
2021-10-30 15                 0.0  0.0  0.0                         19.70   

            

<ipython-input-38-e6e7f90b563c>:11: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  df2=df2.join(df4)


In [39]:
print(df2.columns)

MultiIndex([(                              'Date',     ''),
            (                         'Time_Slot',     ''),
            (                      'Rain Fall mm', 'mean'),
            (                      'Rain Fall mm',  'min'),
            (                      'Rain Fall mm',  'max'),
            (     'Ambient relative humidity %RH', 'mean'),
            (     'Ambient relative humidity %RH',  'min'),
            (     'Ambient relative humidity %RH',  'max'),
            ('Ambient temperature Deg Centigrade', 'mean'),
            ('Ambient temperature Deg Centigrade',  'min'),
            ('Ambient temperature Deg Centigrade',  'max'),
            (            'Ultraviolet index unit', 'mean'),
            (            'Ultraviolet index unit',  'min'),
            (            'Ultraviolet index unit',  'max'),
            (                'Wind velocity km/h', 'mean'),
            (                'Wind velocity km/h',  'min'),
            (                'Wind veloc

In [40]:
Temp_Date = pd.to_datetime(df2["Date"])
df2['Date'] = Temp_Date.dt.strftime('%d-%b-%y')
print(df2.dtypes)
#df2.insert(1,"Time_Slot",df["Time_Slot"])
D=pd.to_datetime(df2["Date"])
year=str(D[0].year)
df2.insert(2,"Crop Year","CY"+year[2:4])

Date                                         object
Time_Slot                                     int64
Rain Fall mm                        mean    float64
                                    min     float64
                                    max     float64
Ambient relative humidity %RH       mean    float64
                                    min     float64
                                    max     float64
Ambient temperature Deg Centigrade  mean    float64
                                    min       int64
                                    max       int64
Ultraviolet index unit              mean    float64
                                    min       int64
                                    max       int64
Wind velocity km/h                  mean    float64
                                    min     float64
                                    max     float64
Wind Cardinal Direction             mode     object
dtype: object


In [41]:
import numpy as np
min_date=min(Temp_Date)
X= (Temp_Date-(min_date))
Week=X//np.timedelta64(1,'W')+1
Week="Week "+Week.astype('str')
df2.insert(2,"Week",Week)
print(Temp_Date.dt.month_name())
df2.insert(3,"Month-Year Mode",Temp_Date.dt.month_name())
df2.insert(4,"Year",Temp_Date.dt.year)
print(Temp_Date.dt.year)

0        October
1        October
2        October
3        October
4        October
          ...   
1086    February
1087    February
1088    February
1089    February
1090    February
Name: Date, Length: 1091, dtype: object
0       2021
1       2021
2       2021
3       2021
4       2021
        ... 
1086    2022
1087    2022
1088    2022
1089    2022
1090    2022
Name: Date, Length: 1091, dtype: int64


In [42]:
df2["Radiation (Mj/M2)","mean"]=df2["Ultraviolet index unit","mean"]/40
df2["Radiation (Mj/M2)","min"]=df2["Ultraviolet index unit","min"]/40
df2["Radiation (Mj/M2)","max"]=df2["Ultraviolet index unit","max"]/40


df2.drop(columns="Ultraviolet index unit",level=0,inplace=True)
output_file_name= file_name+"_one_hourly_aggregate_data.csv"
df2.to_csv(output_file_name,index=False)